<a href="https://colab.research.google.com/github/brucker3/CS5560_proejct/blob/main/Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modelling

In [1]:
#installing pyLDAvis and gensim
!pip install -U pyLDAvis
!pip install -U gensim

Requirement already up-to-date: pyLDAvis in /usr/local/lib/python3.7/dist-packages (3.3.1)
Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [2]:
#importing Libraries needed for completion of topic modeling
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import numpy as np
import time
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.models import CoherenceModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecate

In [3]:
#read the text file 
# with open('/content/SpanishArticle.txt', 'r') as f:
#     text = f.readlines()
with open('./SpanishArticle.txt', 'r') as f:
    meta = f.readlines()

text = meta[0].split('.')
file_df = pd.DataFrame(text, columns=['text'])
# reviews_df=pd.read_csv('/content/SpanishArticle.txt',error_bad_lines=False)
print(file_df.head(6))
print("----------")
print(file_df.dtypes)
print("----------")
print(file_df.shape)


                                                text
0  Londres (CNN)– El organismo de control del gas...
1   La Comisión Electoral del Reino Unido dijo en...
2   Por lo tanto, continuaremos este trabajo como...
3   Johnson ha sido presionado para que explique ...
4   Los primeros ministros reciben £ 30,000 (US$ ...
5   El líder del Reino Unido se enfrentó a un may...
----------
text    object
dtype: object
----------
(32, 1)


**Text Cleaning and Preprocessing**

In [4]:
def initial_clean(text):
    """
    Function to clean text-remove punctuations, lowercase text etc.
    """
    text = re.sub("[^a-zA-Z ]", "", text)
    text = text.lower()  # lower case text
    text = nltk.word_tokenize(text)
    return (text)

**Identifying and removing stopwords**

In [5]:
stop_words = stopwords.words('english')
# adding some more stop words that doesn't convey much meaning in terms of reviews feel free to extend or reduce this list
stop_words.extend(['news', 'say','use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 
                   'good', 'go', 'get', 'do','took','time','year',
                 'done', 'try', 'many', 'some','nice', 'thank', 'think', 'see', 'rather',
                  'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 
                   'even', 'right', 'line','even', 'also', 'may', 'take', 'come',
                   'new','said', 'like','people'])

In [6]:
def remove_stop_words(text):
     return [word for word in text if word not in stop_words]

**Stemming**

In [7]:
stemmer = PorterStemmer()
def stem_words(text):
    """
    Function to stem words
    """
    try:
        text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1] # no single letter words
    except IndexError:
        pass

    return text

**Applying all functions into one**

In [8]:
def apply_all(text):
    """
    
    """
    return stem_words(remove_stop_words(initial_clean(text)))

print(apply_all)

<function apply_all at 0x7fb6f42dc5f0>


**Tokenization**

In [9]:
t1 = time.time()
file_df['tokenized_text'] = file_df['text'].apply(apply_all)
t2 = time.time()
print("Time to clean and tokenize", len(file_df), "SpanishArticle:", (t2-t1)/60, "min") #Time to clean and tokenize 3209 reviews: 0.21254388093948365 min

print('\n')
print("text with their respective tokenize version:" )
print(file_df.head(5))

Time to clean and tokenize 32 SpanishArticle: 0.0007640719413757324 min


text with their respective tokenize version:
                                                text                                     tokenized_text
0  Londres (CNN)– El organismo de control del gas...  [londr, cnn, el, organismo, de, control, del, ...
1   La Comisión Electoral del Reino Unido dijo en...  [la, comisin, elector, del, reino, unido, dijo...
2   Por lo tanto, continuaremos este trabajo como...  [por, lo, tanto, continuaremo, est, trabajo, c...
3   Johnson ha sido presionado para que explique ...  [johnson, ha, sido, presionado, para, que, exp...
4   Los primeros ministros reciben £ 30,000 (US$ ...  [lo, primero, ministro, reciben, us, de, diner...


**LDA implementation**

In [10]:

#Creating a Gensim dictionary from the tokenized data
tokenized = file_df['tokenized_text']
#Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(tokenized)
#Filtering terms .
dictionary.filter_extremes(no_below=1, no_above=0.8)
#converting the dictionary to a bag of words corpus
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized]
print(corpus[:1])

print([[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 4), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)]]
[[('acusacion', 1), ('bori', 1), ('bretaa', 1), ('cnn', 1), ('control', 1), ('corrupcin', 1), ('dado', 1), ('de', 4), ('del', 3), ('departamento', 1), ('down', 1), ('el', 1), ('en', 1), ('enfrenta', 1), ('escndalo', 1), ('financiacin', 1), ('formal', 1), ('gasto', 1), ('gran', 1), ('iniciar', 1), ('investigacin', 1), ('johnson', 1), ('londr', 1), ('ministro', 1), ('mltipl', 1), ('organismo', 1), ('poltico', 2), ('primer', 1), ('que', 1), ('renovacion', 1), ('se', 1), ('sobr', 1), ('street', 1), ('una', 1)]]


In [11]:
#choosing hyperparameter values; num_topics=10, alpha=0.05, eta=0.61, num_words=5 
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=7, alpha=0.05, eta=0.61)

#saving the model
ldamodel.save('model_combined.gensim')
topics = ldamodel.print_topics(num_words=5)
print('\n')
print("Now printing the topics and their composition")
print("This output shows the Topic-Words matrix for the 10 topics created and the 5 words within each topic")
for topic in topics:
   print(topic)




Now printing the topics and their composition
This output shows the Topic-Words matrix for the 10 topics created and the 5 words within each topic
(0, '0.007*"est" + 0.007*"en" + 0.007*"para" + 0.007*"se" + 0.007*"mircol"')
(1, '0.023*"del" + 0.020*"de" + 0.014*"que" + 0.008*"una" + 0.008*"johnson"')
(2, '0.016*"de" + 0.013*"que" + 0.013*"lo" + 0.013*"el" + 0.013*"prstamo"')
(3, '0.040*"de" + 0.023*"del" + 0.021*"que" + 0.015*"el" + 0.012*"su"')
(4, '0.045*"de" + 0.025*"que" + 0.024*"el" + 0.014*"para" + 0.014*"ministro"')
(5, '0.016*"de" + 0.013*"se" + 0.009*"que" + 0.009*"su" + 0.009*"un"')
(6, '0.021*"que" + 0.021*"el" + 0.015*"en" + 0.012*"de" + 0.012*"se"')
(7, '0.011*"comentario" + 0.007*"de" + 0.007*"el" + 0.007*"lder" + 0.007*"en"')
(8, '0.010*"su" + 0.010*"han" + 0.010*"el" + 0.006*"acusacion" + 0.006*"comportamiento"')
(9, '0.013*"para" + 0.009*"que" + 0.009*"est" + 0.009*"el" + 0.009*"en"')


In [12]:
# Compute Coherence Score for num_topics=10, alpha=0.05, eta=0.61, num_words=5
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3575608026648031


In [14]:
#visualizing topics for alpha=0.05, eta=0.61
lda_viz = gensim.models.ldamodel.LdaModel.load('/content/model_combined.gensim')
lda_display = pyLDAvis.gensim_models.prepare(lda_viz, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)